## Encode alignment block from a MAF file

In [1]:
import pandas as pd
from collections import defaultdict

In [8]:
maf_dir = '/lustre/groups/epigenereg01/workspace/projects/vale/vae_effect_prediction/data/extract_genes/protein_coding/genes/0/maf/'

maf_file = maf_dir + 'ENSG00000248405.maf'

maf_file = '/lustre/groups/epigenereg01/workspace/projects/vale/vae_effect_prediction/data/clinvar/noncoding_snps/benign/mafs/0/id_1091621_chr14_96856256.maf'
#maf = pd.read_csv(maf_file, comment='#', sep='\t', names=['block','seq_name','start', 'n_bases', 'strand', 'contig_len', 'seq'])

In [9]:
GAPS_TOL = 50

In [10]:
#c=0

#cs=set()

msa = defaultdict(list)

current_block = defaultdict(list)

c=0

with open(maf_file, 'r') as f:
    
    while True:
        
        line = f.readline()
        
        if line.startswith('s'):
            
            _, contig_name, start, seq_len, orient, _, seq = line.split()
            
            start, seq_len = int(start), int(seq_len)
            
            if  contig_name.startswith('Homo_sapiens'): #first sequence in the alignment block=reference sequence (Homo sapiens)
                human_current_pos = start #human start pos for the current block
                block_len = len(seq) #width of the current block 
                c+=1
                if c%100==0:
                    print(start)
                if not msa:
                    human_start_pos = human_current_pos #if that's the first alignment block in the MAF file
                    #human_contig_name = contig_name
                    
            seq = seq.upper() #ignore strand
            
            current_block[contig_name].append((seq, start, seq_len, orient)) #(sequence, start pos for this sequence in the block, sequence length, orientation)
            #cs.add(seq_name)
            
        elif (current_block and line.startswith('a')) or line=='': #start of the new alignment block or EOF
            
            #process previous block
            
            #c+=1
            
            updated_contigs = defaultdict(set) #sequences indices within contigs updated by the block

            for contig_name in current_block.keys():
                
                gaps_len = []
                
                for seq_idx, (seq, start, seq_len, orient) in enumerate(current_block[contig_name]): #loop over fragments in the block
                    for old_seq_idx, (old_seq, old_orient, old_start, next_expected_pos) in enumerate(msa[contig_name]): #loop over already existing sequences
                        gap_len = start-next_expected_pos #gap length if added to the current sequence
                        if gap_len>=0 and gap_len<GAPS_TOL and old_orient==orient:
                            gaps_len.append((seq_idx, old_seq_idx, gap_len))
                
                gaps_len.sort(key=lambda x:x[2]) #sort by gap_len
                
                seq_idx_to_exclude, old_seq_idx_to_exclude = [], []
                
                for seq_idx, old_seq_idx, _ in gaps_len:
                    #add fragments to already existing sequences starting from the minimal gap pair
                    if seq_idx not in seq_idx_to_exclude and old_seq_idx not in old_seq_idx_to_exclude:
                        
                        seq, start, seq_len, _ = current_block[contig_name][seq_idx]
                        next_expected_pos = start+seq_len
                        msa[contig_name][old_seq_idx][3] = next_expected_pos #expected start position for the next fragment of this sequence
                        msa[contig_name][old_seq_idx][0] += seq #extend the seqeunce with the fragment
                        updated_contigs[contig_name].add(old_seq_idx) #mark the update
                        
                        seq_idx_to_exclude.append(seq_idx) #don't use this fragment anymore
                        old_seq_idx_to_exclude.append(old_seq_idx) #don't add to this sequence anymore
                        
                for seq_idx, (seq, start, seq_len, orient) in enumerate(current_block[contig_name]):
                    if not seq_idx in seq_idx_to_exclude:
                        #add new sequence
                        gap_len = human_current_pos-human_start_pos #insert gaps from the beginning of the alignment to the start of the sequence 
                        gaps = '-'*gap_len 
                        next_expected_pos = start+seq_len #expected start position for the next fragment of this sequence
                        seq = gaps+seq
                        msa[contig_name].append([seq, orient, start, next_expected_pos]) #start new sequence with the current fragment
                        updated_contigs[contig_name].add(len(msa[contig_name])-1) #mark the update

            for contig_name, subseq in msa.items():
                #add gaps to all sequences that were not in the current block
                missing_seq_idx = set(range(len(subseq)))-updated_contigs[contig_name]
                for seq_idx in missing_seq_idx:
                    msa[contig_name][seq_idx][0] += '-'*block_len

            current_block = defaultdict(list) #empty the block to collect the next one
            
        if line == '':
            break
            #cs=set()
        

96856293
96856488


In [11]:
#convert msa to a dataframe

msa_list = []

for full_contig_name in msa.keys():
    if 'Homo_sapiens' in full_contig_name:
        species_name = f'{full_contig_name}/{human_start_pos}-{msa[full_contig_name][0][-1]}'
    else: 
        species_name = full_contig_name.split('.')[0] #separate species name from full contig name
    contig_name = '.'.join(full_contig_name.split('.')[1:])
    for subseq, orient, start, end in msa[full_contig_name]: #end corresponds to the next expected position for this subsequence (i.e. not inclusive)
        msa_list.append((species_name, contig_name, start, end, subseq, orient)) 
        
msa_df = pd.DataFrame(msa_list, columns=['species', 'contig', 'start', 'end', 'seq', 'orient'])

#msa_df['species'] = msa_df.full_contig_name.apply(lambda x: x.split('.')[0])
#msa_df['contig'] = msa_df.full_contig_name.apply(lambda x: '.'.join(x.split('.')[1:]))

In [12]:
def hamming_distance(s1, s2):
    """Return the Hamming distance between equal-length sequences"""
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 != ch2  for ch1, ch2 in zip(s1, s2)) 

In [13]:
ref_seq = msa_df.iloc[0].seq

msa_df['hamming'] = msa_df.seq.apply(lambda seq:hamming_distance(seq, ref_seq)) #add hamming distance between Homosapiens and all other sequences

msa_df.iloc[1:] = msa_df.iloc[1:].sort_values(by=['species', 'hamming', 'contig']) #sort elverything except the 1st line (Homo sapiens)

In [14]:
msa_df['overlap_idx'] = [[] for row_idx in range(len(msa_df))]

#detect overlapping subsequences within  the same contig: very rare event!
for species in msa_df.species.unique():
    species_df = msa_df[msa_df.species == species][['contig', 'start', 'end']]
    for contig in species_df.contig.unique():
        contig_coords = species_df[species_df.contig == contig][['start', 'end']].reset_index().values
        for idx_s1, start_s1, end_s1 in contig_coords:
            for idx_s2, start_s2, end_s2 in contig_coords:
                if idx_s2!=idx_s1:
                    if ((start_s1>=start_s2 and start_s1<end_s2) or (start_s2>=start_s1 and start_s2<end_s1) 
                        or (end_s1>start_s2 and end_s1<=end_s2) or (end_s2>start_s1 and end_s2<=end_s1)):
                        msa_df.loc[idx_s1, 'overlap_idx'].append(idx_s2)
                        msa_df.loc[idx_s2, 'overlap_idx'].append(idx_s1)
                        print('Overlap detected', species, contig)

In [15]:
unique_seq = []

#construct a single sequence per species by performing netting
#we choose the sequence with the smallest hamming distance as the reference
#we fill gaps in the reference sequence using sequences with larger and larger hamming distance
for species in msa_df.species.unique():
        idx_to_exclude = []
        species_df = msa_df[msa_df.species == species][['seq','overlap_idx']]
        species_seq = list(species_df.iloc[0].seq)
        for idx, (seq, overlap_idx) in species_df.iloc[1:].iterrows():
            if idx not in idx_to_exclude:
                idx_to_exclude.extend(overlap_idx) #we won't consider any sequences that overlap the current sequence
                species_seq = [species_seq[i] if species_seq[i]!='-' else seq[i] for i in range(len(species_seq))] #fill gaps in the refseq using current sequence
        species_seq = ''.join(species_seq)
        unique_seq.append((species,species_seq,hamming_distance(species_seq, ref_seq)))

In [ ]:
#print MSA in a2m format

for species, seq, *_ in unique_seq:
    seq = ''.join([s if s in 'ACTG' else '-' for s in seq])
    print('>'+species)
    print(seq)

In [18]:
#visualize aligned sequences

#species_max_length=max([len(s) for s in msa_df.species.unique()])

R=25

refseq = unique_seq[0][1][250-R:250+R]

for species, seq, *_ in unique_seq:
    seq=list(seq[250-R:250+R])
    for c_ind in range(len(seq)):
            #highligh where the base is different from reference
            seq[c_ind] = (seq[c_ind] if seq[c_ind]==refseq[c_ind] else "\x1b[31m{}\x1b[0m".format(seq[c_ind]))#dye the site at mismatches in red using ANSI escape code
    seq = ''.join(seq)
    
    print(f'{species:<40}{seq}')


Homo_sapiens.chr14/96856006-96856506    TTAGAGATTCCAAAATTAGGTAAAGGAAAACTTAAGTTATTTCTAGCAAA
Acanthisitta_chloris                    TCAGAGACTCAAAGATCAGGTGAGGAACTACTTCTTTGTTTTT-----AA
Acinonyx_jubatus                        TTAGAGATTCCAAAATTAGGTAAAGGAAAACT-A-----TTTT-GGTAAA
Acomys_cahirinus                        TTAGGGACTCCAAAATTAGGTAAG-AACAACTTACATTATTTT-GGTTGG
Acrocephalus_arundinaceus               TCAGAGACTCAAAAATCAGGTGAGAAACTATTTCATTCCTTCT-----AA
Aegithalos_caudatus                     TCAGAGACTCAAAAATCAGGTGAGAAACTATTTCATTCCTTTT-----AA
Aegotheles_bennettii                    TCAGAGACTCAAAAATCAGGTGAGAAACTACTTCTTTCCTTTG-----GA
Agapornis_roseicollis                   TCAGAGACTCAAAAATTGGGTGAGAAGCTACTTCTTTCCATTT-----AA
Agelaius_phoeniceus                     TCAGAGACTCAAAAATCAGGTGAGAAGCTATTTCATTCCTTTT-----AA
Ailuropoda_melanoleuca                  TTAGAGATTCCAAAATTAGGTAAAGGAAAACT-AAATTATTTT-GGCAAA
Ailurus_fulgens                         TTAGAGATTCCAAAATTAGGTAAAGGAAAACT-AAATTATTTT-GGCAAA

In [24]:
msa_df = pd.DataFrame(unique_seq, columns=['species', 'seq', 'hamming'])

msa_df

,species,seq,hamming
0,Homo_sapiens.chr1/110603452-110603952,GGTCAGGGGAGGATGGGATCTTTGGACAGCTTGTCACTTGCAAGTA...,0
1,Acanthisitta_chloris,GGTCAGGGGAAGAAGGGATCTTTGGGCAGCTGGTTACTTGCAGATA...,88
2,Acinonyx_jubatus,GGTCAGGGGAGGATGGGATCTTTGGACAGCTCGTCACTTGCAGGTA...,26
3,Acomys_cahirinus,GGTCAGGGGAGGACGGGATCTTTGGACAGCTTGTCACTTGCAAGTA...,253
4,Acrocephalus_arundinaceus,GGTCAGGGGAAGAGGGGATCTTTGGGCAGCTGGTTACTTGCAAATA...,86
...,...,...,...
599,Zapus_hudsonius,GGTCAGGGGAGGATGGGATCTTTGGACAGCTTGTCACTTGCAAGTA...,249
600,Ziphius_cavirostris,GGTCAGGGGAGGATGGGATCTTTGGACAGCTCGTCACTTGCAAGTA...,24
601,Zonotrichia_albicollis,---------------------TTGGGCAGCTGGTTACTTGCAAATA...,101
602,Zosterops_hypoxanthus,GGTCAGGGGAAGAGGGGATCTTTGGGCAGCTGGTTACTTGCAAATA...,83


In [158]:
#quick test approach to MAF file conversion
#doesn't take multiple seqeunces from the same contig into account

# msa = defaultdict(str)

# current_block_seq = set()

# with open(maf_file, 'r') as f:
#     for line in f:
#         if line.startswith('s'):
#             _, seq_name, start, _, _, _, seq = line.split()
#             if  seq_name.startswith('Homo_sapiens'): #first seqeunce in the alignment block=reference sequence
#                 human_current_pos = int(start)
#                 seq_len = len(seq) #alignment length for the current block 
#                 if not msa:
#                     human_start_pos = human_current_pos #if that's the first alignment block in the MAF file
#             seq = seq.upper() #ignore strand
#             if seq_name not in msa.keys():
#                 msa[seq_name] = '-'*(human_current_pos-human_start_pos) #add gaps before the sequence start
#             current_block_seq.add(seq_name) #collect sequences for the current block
#             msa[seq_name] += seq  #add current sequence to the alignment
#         elif msa and line.startswith('a'):
#             seq_with_gaps = set(msa)-current_block_seq #seqences that are known but were absent in the last alignment block
#             for seq_name in seq_with_gaps:
#                 msa[seq_name] += '-'*seq_len #add gaps to these sequences for the length of the last alignment block
#             current_block_seq = set() 